In [12]:
import os
import tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import tensorflow as tf
from tfx.components import (
    CsvExampleGen, 
    StatisticsGen, 
    SchemaGen, 
    ExampleValidator, 
    Transform,
    Tuner, 
    Trainer,
    Evaluator,
    Pusher
)

from tfx.proto import trainer_pb2, example_gen_pb2, pusher_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 

import tensorflow_model_analysis as tfma 
PIPELINE_NAME = "alfitoptr-loan-pipeline"
 
# pipeline inputs
DATA_ROOT = "data"
TRANSFORM_MODULE_FILE = "modules/loan_transform.py"
TRAINER_MODULE_FILE = "modules/loan_trainer.py"

OUTPUT_BASE = "output"
serving_model_dir = os.path.join(OUTPUT_BASE, 'serving_model')
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")

interactive_context = InteractiveContext(pipeline_root=pipeline_root)

Kode tersebut mendefinisikan elemen-elemen utama untuk membangun pipeline dengan TensorFlow Extended (TFX). ```PIPELINE_NAME``` adalah nama pipeline utama, ```pipeline_root``` menentukan lokasi penyimpanan artefak pipeline, sedangkan ```metadata_path``` adalah jalur database SQLite untuk menyimpan metadata pipeline. SERVING_MODEL_DIR digunakan untuk menyimpan model yang telah dilatih dan siap digunakan (serving), sementara ```DATA_ROOT``` adalah direktori tempat data mentah berada. Objek ```InteractiveContext``` memungkinkan pengembangan pipeline secara interaktif dengan menyimpan artefak di jalur yang ditentukan oleh ```pipeline_root```.

In [3]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [4]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

Kode ini mengimpor data CSV, memprosesnya, dan membaginya menjadi data pelatihan (train) dan evaluasi (eval) dalam proporsi 80:20 untuk digunakan dalam pipeline TFX.

Kode tersebut membuat konfigurasi untuk pembagian data (data splitting) dan menjalankan komponen `CsvExampleGen ` menggunakan TFX secara interaktif.
1. **output**:
    - Membuat konfigurasi keluaran untuk komponen `ExampleGen`.
    - Menggunakan `example_gen_pb2.Output` untuk mendefinisikan pembagian data (split configuration).
    - Data dibagi menjadi dua bagian:
        - Train: 80% (8 hash buckets).
        - Eval: 20% (2 hash buckets).
2. **example_gen**:
    - Membuat komponen `CsvExampleGen` untuk membaca data CSV dari direktori yang ditentukan (DATA_ROOT).
    - Parameter output_config diteruskan ke komponen ini untuk menentukan konfigurasi pembagian data.
3. **interactive_context.run(example_gen)**:
    - Menjalankan example_gen secara interaktif.
    - Proses ini membaca data mentah dari DATA_ROOT, membagi data sesuai konfigurasi, dan menghasilkan artefak data terproses yang digunakan dalam pipeline berikutnya.

In [5]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
 
 
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [6]:
interactive_context.show(statistics_gen.outputs["statistics"])

Kode ini bertujuan untuk menghasilkan statistik deskriptif dari data dan menampilkan hasilnya menggunakan komponen `StatisticsGen` dari TFX. 

1. **statistics_gen**:
    - Membuat komponen StatisticsGen yang bertugas menghasilkan statistik deskriptif dari dataset.
    - Parameter examples mengambil keluaran data yang telah diproses oleh CsvExampleGen (example_gen.outputs["examples"]).
2. **interactive_context.run(statistics_gen)**:
    - Menjalankan komponen StatisticsGen secara interaktif.
    - Komponen ini membaca data terproses dari artefak yang dihasilkan oleh example_gen dan menghitung statistik, seperti distribusi nilai, nilai kosong, dan outlier.
3. **interactive_context.show(statistics_gen.outputs["statistics"])**:
    - Menampilkan statistik deskriptif hasil proses StatisticsGen.

In [7]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"])
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [8]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'loan_amnt',FLOAT,required,,-
'loan_int_rate',FLOAT,required,,-
'loan_percent_income',FLOAT,required,,-
'loan_status',INT,required,,-
'person_home_ownership',INT,required,,-
'person_income',FLOAT,required,,-
'previous_loan_defaults_on_file',INT,required,,-


Kode ini menggunakan SchemaGen untuk menghasilkan skema data berdasarkan statistik yang telah dihitung oleh `StatisticsGen`. Skema ini menggambarkan struktur dataset, termasuk nama fitur, tipe data (numerik atau kategorikal), rentang nilai, dan aturan validasi seperti apakah fitur wajib atau opsional. Dengan menjalankan `interactive_context.run(schema_gen)`, pipeline menghasilkan skema, yang kemudian ditampilkan menggunakan `interactive_context.show(schema_gen.outputs["schema"])`.

In [9]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [10]:
interactive_context.show(example_validator.outputs['anomalies'])

Kode ini menggunakan komponen `ExampleValidator` untuk memvalidasi data dengan membandingkannya terhadap skema yang dihasilkan oleh `SchemaGen` dan statistik yang dihitung oleh `StatisticsGen`. Dengan menjalankan interactive_context.run(example_validator), komponen ini memeriksa apakah data sesuai dengan aturan yang ditetapkan dalam skema, seperti memastikan nilai berada dalam rentang yang benar dan tipe data yang sesuai. Hasil validasi, yang berupa anomali atau ketidaksesuaian data, ditampilkan menggunakan interactive_context.show(example_validator.outputs['anomalies']), membantu mengidentifikasi masalah dalam data sebelum diproses lebih lanjut dalam pipeline TFX. 

Terlihat hasil output tidak terdapat anomali yang terdeteksi

In [11]:
TRANSFORM_MODULE_FILE = "modules/loan_transform.py"
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],    
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: output\alfitoptr-loan-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\c736e357ffbc47f6bdddb480d515e879\assets


INFO:tensorflow:Assets written to: output\alfitoptr-loan-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\c736e357ffbc47f6bdddb480d515e879\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

kode diatas menjalankan file transformasi yang telah dibuat terhadap data. Transformasi yang dilakukan adalah standarisasi terhadap data respon numerik dan one hot encoding terhadap data respon kategorik.

In [13]:
TRAINER_MODULE_FILE = "modules/loan_trainer.py"
 
tuner = Tuner(
        module_file=os.path.abspath(TRAINER_MODULE_FILE),
        examples=transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        schema=schema_gen.outputs['schema'],
        train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=5000),
        eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=1000)
        )

In [14]:
interactive_context.run(tuner)

Trial 3 Complete [00h 00m 24s]
val_binary_accuracy: 0.9130468964576721

Best val_binary_accuracy So Far: 0.9130468964576721
Total elapsed time: 00h 01m 13s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in output/tuning_resuls\untitled_project
Showing 10 best trials
Objective(name="val_binary_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
unit1: 64
unit2: 96
unit3: 32
Score: 0.9130468964576721

Trial 0 summary
Hyperparameters:
unit1: 32
unit2: 64
unit3: 96
Score: 0.9117812514305115

Trial 1 summary
Hyperparameters:
unit1: 64
unit2: 32
unit3: 96
Score: 0.9065155982971191


ExecutionResult(
    component_id: Tuner
    execution_id: 6
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={})
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}))

Kode diatas menjalankan tuner yang ada dalam file `TRAINER_MODULE_FILE`. Tuning parameter dilakukan menggunakan metode `RandomSearch` terhadap jumlah neuron masing masing layer. Parameter yang diuji dalam rentang 32, 64, 98, 126, dengan Training Steps = 5000 dan Validation steps = 1000. Didapat model terbaik dengan akurasi 0.913 dengan neuron pada layer pertama adalah 64, layer kedua adalah 96, dan 32 untuk neuron pada layer ketiga

In [19]:
trainer  = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(),
    eval_args=trainer_pb2.EvalArgs(),
)

interactive_context.run(trainer)

INFO:tensorflow:Reloading Tuner from output/tuning_resuls\untitled_project\tuner0.json


INFO:tensorflow:Reloading Tuner from output/tuning_resuls\untitled_project\tuner0.json


INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 person_home_ownership_ohe (Inp  [(None, 4)]         0           []                               
 utLayer)                                                                                         
                                                                                                  
 previous_loan_defaults_on_file  [(None, 3)]         0           []                               
 _ohe (InputLayer)                                                                                
                                                                                                  
 person_income (InputLayer)     [(None, 1)]          0           []                               
                                                                                              

...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: output\alfitoptr-loan-pipeline\Trainer\model\9\Format-Serving\assets


INFO:tensorflow:Assets written to: output\alfitoptr-loan-pipeline\Trainer\model\9\Format-Serving\assets


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


ExecutionResult(
    component_id: Trainer
    execution_id: 9
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

Kode diatas menjalankan training model menggunakan model terbaik hasil tuning parameter. Model dilatih dengan 10 epoch, training steps = 5000 dan validation steps = 1000

In [20]:
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 10
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

Kode ini menggunakan komponen Resolver dalam pipeline TFX.  Kode ini digunakan untuk secara otomatis memilih model terbaru yang telah berhasil divalidasi dan diterima sebagai model yang siap diproduksi, yang berguna dalam pipeline TFX untuk memastikan penggunaan model yang paling up-to-date dan telah diuji.

In [21]:
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='loan_status')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs = [
    tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name="Precision"),
            tfma.MetricConfig(class_name="Recall"),
            tfma.MetricConfig(class_name="ExampleCount"),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]
 
)

Kode ini mendefinisikan `EvalConfig` untuk evaluasi model menggunakan TensorFlow Model Analysis (TFMA). Konfigurasi ini mencakup model_specs yang menentukan loan_status sebagai label target, slicing_specs untuk evaluasi model di seluruh dataset tanpa pemotongan khusus, dan metrics_specs yang mencakup berbagai metrik evaluasi, seperti ExampleCount, AUC, Precision, Recall, dan BinaryAccuracy. Untuk BinaryAccuracy, terdapat pengaturan ambang batas, dengan nilai minimum 0.5 untuk metrik tersebut, dan perubahan sekecil 0.0001 yang dianggap signifikan jika skor berubah dalam arah yang lebih baik (higher is better). Evaluasi ini digunakan untuk mengukur kinerja model dalam berbagai metrik yang relevan dalam konteks klasifikasi biner.

In [22]:
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
 
interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 11
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

Kode ini menggunakan Evaluator dari TFX untuk melakukan evaluasi model dalam pipeline. Evaluator membandingkan model yang dilatih (trainer.outputs['model']) dengan model dasar (baseline model) yang dipilih oleh model_resolver (model_resolver.outputs['model']) berdasarkan konfigurasi evaluasi yang telah ditentukan sebelumnya (eval_config). Evaluator menerima data contoh yang dihasilkan oleh example_gen (example_gen.outputs['examples']). Fungsi dari Evaluator ini adalah untuk mengevaluasi model yang dilatih menggunakan berbagai metrik yang telah dikonfigurasi dalam eval_config dan membandingkannya dengan model baseline untuk menentukan apakah model yang baru lebih baik atau tidak. Setelah evaluasi, hasilnya digunakan untuk memutuskan apakah model baru siap diproduksi atau tidak. interactive_context.run(evaluator) menjalankan evaluator dalam konteks interaktif.

In [23]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

Kode tersebut bertujuan untuk menampilkan hasil evaluasi yang telah dibentuk sebelumnya. Didapat AUC: **0.964** Akurasi: **0.912** Precission: **0.805** dan Recall: **0.782**

In [24]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(base_directory=serving_model_dir)
    )
)

In [25]:
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 12
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))

Kode ini menggunakan komponen Pusher dari TFX untuk mendorong model yang telah dilatih ke lokasi penyimpanan yang ditentukan, jika model tersebut lolos evaluasi. Pusher menerima tiga argumen utama: model yang dilatih (trainer.outputs['model']), status keberkahan model yang dihasilkan oleh Evaluator (evaluator.outputs['blessing']), dan tujuan penyimpanan untuk model yang diset di push_destination. Tujuan penyimpanan model adalah direktori `serving_model_dir`, yang diatur menggunakan pusher_pb2.PushDestination.Filesystem. Pusher ini akan memindahkan model ke direktori tersebut jika model dianggap valid. Dengan menjalankan interactive_context.run(pusher), model akan dipindahkan ke folder yang sesuai untuk digunakan dalam tahap produksi atau serving.

In [26]:
!jupyter nbconvert --to script loan_pipeline.ipynb

[NbConvertApp] Converting notebook loan_pipeline.ipynb to script
[NbConvertApp] Writing 12928 bytes to loan_pipeline.py
